# Agents
- AIエージェントを実装するためのモジュール
- ユーザーの入力内容を踏まえて処理の流れを変える
    - 社内文書をVector Storeから検索
    - Web上の情報を検索して回答
    - etc..
- Function Callingとは違う
    - Function Calling機能を持たないモデルでもAgentsは実装可能

In [1]:
# Agentsを使ってコマンドを実行し、指定ディレクトリのファイル一覧を得る
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0,
)

# Bash等のコマンド実行ツールをツールとして準備
tools = load_tools(['terminal'])

# zero-shot-react-descriptionという種類のAgent
agent_chain = initialize_agent(
    tools,
    chat,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

result = agent_chain.run('sample_dataディレクトリにあるファイルの一覧を教えて')
print(result)

/usr/local/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
/usr/local/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.12/site-packages/langchain_community/tools/shell/tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


Executing command:
 ls sample_data
fuga.txt, hoge.txt, ほげほげ.txt


## ReAct
- ReAct = Reasoning ＋ Acting からなる造語
- LLMに　`思考(理由)→行動`　をくり返させ、最終的に精度の高い回答を得ようとするテクニック
- 行動の際に、LLMが持っている（学習済みの）知識だけでなく、その他の情報（Webやツール等）の活用が含まれる
    - Shellコマンドの実行
    - Web検索
    - etc..
- 以下のようなシーケンス
    1. タスクへの出力を生成するために必要な 行動（Act）とその 理由（Reason）を思考する。
    1. この思考をもとに行動し結果を得る
    1. 再度次に必要な行動とその理由を思考する
    1. 上記の思考→行動を繰り返し、ファイナルアンサーが生成できた時点で終了する


#### ディレクトリにあるファイル一覧を取得した例
1. `思考→行動→観測` を繰り返すことと `使用できるツール` をLLMに投げる
    ```
    Answer the following question as best you can. You have access to the following tools:

    terminal: Run shell commands on this Linux machine.

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [terminal]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now knwo the final answer
    Final Answer: the final answer to the original input question

    Begin!

    Question: sample_dataディレクトリにあるファイルの一覧を教えて
    Thought:
    ```
1. LLMから思考(理由) + 行動が応答される
    ```
    I need to list the files in the sample_data directory.
    Action: terminal
    Action Input: ls sample_data
    ```
1. LangChainのAgentが行動方法(ツール)とInputを抽出
    - 行動方法: terminal
    - Input: ls sample_data

1. LangChainのAgentがツールを実行
    - ls sample_data を実行しファイル一覧を得る

1. LLMに行動結果を追加したプロンプトを投げる
    ```
    Answer the following question as best you can. You have access to the following tools:

    terminal: Run shell commands on this Linux machine.

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [terminal]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now knwo the final answer
    Final Answer: the final answer to the original input question

    Begin!

    Question: sample_dataディレクトリにあるファイルの一覧を教えて
    Thought: I need to list the files in the sample_data directory.
    Action: terminal
    Action Input: ls sample_data
    Observation: fuga.txt
    hoge.txt
    ほげほげ.txt

    Thought:
    ```

1. LLMから思考と最終回答が応答される
    ```
    I now know the files in the sample_data directory.
    Final Answer: fuga.txt, hoge.txt, ほげほげ.txt
    ```

1. LangChainのAgentがFinal Answer: の箇所を最終的な回答として抽出

## Tools
- 以下のようなツールが使用可能
| Tool          | 概要                   |
| ------------- | ---------------------- |
| terminal      | シェルでコマンドを実行 |
| Python_REPL   | Pythonのコードを実行   |
| google_search | Googleで検索する       |
| Wikipedia     | Wikipediaを検索する    |
| human         | 人間に入力させる       |

- 実態は単なるPythonの関数
    - 自作も可能
    - 任意の関数をツールとして指定できるので、Chainsを指定することも可能
- ツール次第で任意のコマンド・プログラム・SQL等が実行できてしまう
    - 危険なので使用時は要注意
    - 実行前に人間がチェックする Human-in-the-loopを使うのも手
        - HumanApprovalCallbackHandler

In [3]:
# 文章要約のChainをToolsで指定する例
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain_openai import ChatOpenAI

# プロンプト
template = """
以下の文章の結論だけ一言に要約してください。

{input}
"""
summarize_prompt = PromptTemplate(
    input_variables=['input'],
    template=template,
)

# LLM
chat = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0,
)

# Chain
summerize_chain = LLMChain(
    prompt=summarize_prompt,
    llm=chat,
)

# ツール定義
summary_tool = Tool.from_function(
    func=summerize_chain.run,
    name='Summarizer',
    description='Text summarizer'
)

# リストにまとめてagentに渡せるようにする
tools = [
    summary_tool,
]

## Toolkits
- 同時に使うツールを一つにまとめたもの
- 以下、一例
| Toolkits                      | 含まれるツール                                     |
| ----------------------------- | -------------------------------------------------- |
| AzureCognitiveServicesToolkit | Azure Cognitive ServiceのAPIのいくつかの機能       |
| GmailToolkit                  | Gmailでのメール検索や送信                          |
| JiraToolkit                   | Jiraでの課題の検索や作成                           |
| O365Toolkit                   | Office 365のカレンダーの検索やメールの送信         |
| OpenAPIToolkit                | OpenAPI仕様(Swagger)に従った各種API操作            |
| SQLDatabaseToolkit            | DBのスキーマの取得やSQL実行                        |
| VectorStoreToolkit            | Vector Storeの単なる検索やソース(情報源)付きの検索 |
| ZapierToolkit                 | ZapierのNatural Language Actionsの各種アクション   |

## Function callingを使うOpenAI Functions Agent
- ReAct等でAgentsを安定的に動かすのは簡単ではない
    - zero-shot-react-dexcription　では、、、
        - フォーマットに従った回答が得られずエラー、、、など
- Function callingを使ったAgentを使用すると動作が安定しやすい
    - LLMがFunction callingに対応している必要はある

In [14]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import ChatOpenAI

# LLM
chat = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0,
)

# ツール指定
tools = load_tools(['terminal'])

# Agent生成
agent_chain = initialize_agent(
    tools=tools,
    llm=chat,
    agent=AgentType.OPENAI_FUNCTIONS  # ★ポイント: 内部的にOpen AI APIのfunction calling機能を使ってLLMとやり取りしていると思われる
)

# 実行
result = agent_chain.run('sample_dataディレクトリにあるファイル一覧を教えて')
print(result)

/usr/local/lib/python3.12/site-packages/langchain_community/tools/shell/tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


Executing command:
 ['ls sample_data']
`sample_data`ディレクトリには以下のファイルがあります:
- fuga.txt
- hoge.txt
- ほげほげ.txt


## Open AI Multi Function Agent
- 一度に複数のツールを使う
- LLMからの返答時に下記のような複数ツールの実行希望が返る
    - Web検索関数: 東京の天気
    - Web検索関数: 大阪の天気
- 受け取った側も並列してツールの実行が可能
- その結果
    - LLMとのやり取りの回数が減る
    - ツール実行が効率化される
    - 処理速度向上！

In [6]:
# 登録無しで使える検索ツールを例に使う
!pip install duckduckgo-search

In [5]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import ChatOpenAI

# LLM
chat = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0,
)

# ツール指定
tools = load_tools([
    'ddg-search',
])

# Agent生成
agent_chain = initialize_agent(
    tools=tools, 
    llm=chat, 
    agent=AgentType.OPENAI_MULTI_FUNCTIONS,  # ★ポイント
)

# 実行
result = agent_chain.run('東京と大阪の天気を教えて')
print(result)

東京の天気情報は、今日は雨が降り、気温が低くなるようです。寒さ対策をしてお出かけすると良いでしょう。

大阪の天気情報では、今日は小雨や曇りが続き、明日は晴れがちで気温は9度前後になるようです。お出かけの際は天気に注意してください。


## まとめ
- Agentを使うとLLMにどのツールを使うか選択させながら動作させられる
    - Chainsでは固定された処理だけだった
- 柔軟な応答や処理が求められる場合の選択肢となりそう